<h1 style="text-align:center;">Cameron Hicks - Capstone Project Two</h1>
<h1 style="text-align:center;">Hotel Reservation Cancellation Risk Model</h1>
<h2 style="text-align:center;">Data Wrangling</h2>

## Import all necessary packages

In [60]:
import os
import glob
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import time
from tqdm.notebook import tqdm
from pathlib import Path
import json

## Import the Hotel Bookings Data

In [2]:
# Importing the data

hotel_df = pd.read_csv('/Users/cameronhicks/Documents - On Mac/Git Hub Files/Springboard-Capstone-Project-2/Data/hotel_bookings.csv')

## Inspect the data to learn next steps

I will inspect several different points in this data, including looking at the head, dtypes, info, shape, and how many records there are per hotel

In [3]:
# Inspect the dataset

hotel_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
hotel_df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [5]:
# Which columns are categorical?
for col in hotel_df.columns:
    print(f"{col}: {hotel_df[col].nunique()}")

hotel: 2
is_canceled: 2
lead_time: 479
arrival_date_year: 3
arrival_date_month: 12
arrival_date_week_number: 53
arrival_date_day_of_month: 31
stays_in_weekend_nights: 17
stays_in_week_nights: 35
adults: 14
children: 5
babies: 5
meal: 5
country: 177
market_segment: 8
distribution_channel: 5
is_repeated_guest: 2
previous_cancellations: 15
previous_bookings_not_canceled: 73
reserved_room_type: 10
assigned_room_type: 12
booking_changes: 21
deposit_type: 3
agent: 333
company: 352
days_in_waiting_list: 128
customer_type: 4
adr: 8879
required_car_parking_spaces: 5
total_of_special_requests: 6
reservation_status: 3
reservation_status_date: 926


In [6]:
hotel_df.shape

(119390, 32)

In [7]:
hotel_df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [8]:
hotel_df.info

<bound method DataFrame.info of                hotel  is_canceled  lead_time  arrival_date_year  \
0       Resort Hotel            0        342               2015   
1       Resort Hotel            0        737               2015   
2       Resort Hotel            0          7               2015   
3       Resort Hotel            0         13               2015   
4       Resort Hotel            0         14               2015   
...              ...          ...        ...                ...   
119385    City Hotel            0         23               2017   
119386    City Hotel            0        102               2017   
119387    City Hotel            0         34               2017   
119388    City Hotel            0        109               2017   
119389    City Hotel            0        205               2017   

       arrival_date_month  arrival_date_week_number  \
0                    July                        27   
1                    July                        27  

In [9]:
# How many reservations are recorded for each hotel?
total_res_counts = hotel_df['hotel'].value_counts()

# How many of these reservations were canceled
canceled_counts = hotel_df.groupby('hotel')['is_canceled'].sum()

# Combine in a dataframe to view totals and total canceled by hotel
counts_summary = pd.DataFrame({'Total Reservations': total_res_counts, 'Cancellations': canceled_counts})

print(counts_summary)

              Total Reservations  Cancellations
hotel                                          
City Hotel                 79330          33102
Resort Hotel               40060          11122





## Missing Values

After inspecting the dataset it is time to handle any missing values prior to importing information about weather. I will find the missing values, then explore as necessary to determin the best way to deal with theml



In [10]:
# Begin to inspent and handle missing values

hotel_df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

It is a safe assumption that null values for children means the reservation did not include any children. Filling those null values with 0

In [11]:
# Filling null children with 0

hotel_df['children'].fillna(0, inplace=True)

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/2854115428.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hotel_df['children'].fillna(0, inplace=True)


Countries may have a correlation with the cancelations, however there is not a logical way to determine the missing values. It will be best to fill them with "Unknown" so we don't lose important data. We may determine later that we need to drop the unknown country records.

In [12]:
# Filling null countries with 'Unknown'

hotel_df['country'].fillna('Unknown', inplace=True)

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/1703432613.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hotel_df['country'].fillna('Unknown', inplace=True)


In [13]:
hotel_df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               0
babies                                 0
meal                                   0
country                                0
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

Agent is a bit more complex. It is possible that agent will have a correlation with cancellations, and also very likely that the null values simply didn't book with an agent. I need to know more to decide how to handle these nulls.

In [14]:
# Further inspecting agent to decide how to handle null values

agent_summary = (hotel_df.groupby(['hotel', 'agent']).agg(tot_agent_res = ('is_canceled', 'count'), tot_agent_cancel = ('is_canceled', 'sum')).reset_index())

agent_summary['cancellation_rate'] = (agent_summary['tot_agent_cancel'] / agent_summary['tot_agent_res'])

agent_summary.sort_values(by=['hotel', 'tot_agent_cancel'], ascending=[True, False], inplace=True)

agent_summary.head(20)

,hotel,agent,tot_agent_res,tot_agent_cancel,cancellation_rate
8,City Hotel,9.0,31955,13262,0.415021
0,City Hotel,1.0,7137,5230,0.732801
5,City Hotel,6.0,2683,966,0.360045
17,City Hotel,19.0,1061,780,0.735156
2,City Hotel,3.0,1308,767,0.586391
35,City Hotel,37.0,1230,717,0.582927
13,City Hotel,14.0,3640,652,0.179121
27,City Hotel,29.0,679,544,0.801178
19,City Hotel,21.0,875,506,0.578286
140,City Hotel,229.0,786,484,0.615776


In [15]:
agent_summary.sort_values(by=['hotel', 'cancellation_rate'], ascending=[True, False], inplace=True)

agent_summary.head(20)

,hotel,agent,tot_agent_res,tot_agent_cancel,cancellation_rate
144,City Hotel,236.0,247,247,1.0
178,City Hotel,326.0,160,160,1.0
4,City Hotel,5.0,102,102,1.0
115,City Hotel,170.0,93,93,1.0
39,City Hotel,41.0,75,75,1.0
163,City Hotel,286.0,45,45,1.0
112,City Hotel,162.0,37,37,1.0
65,City Hotel,78.0,29,29,1.0
143,City Hotel,235.0,29,29,1.0
43,City Hotel,47.0,26,26,1.0


In [16]:
hotel_df['agent'].max()

535.0

In [17]:
hotel_df['agent'].min()

1.0

After inspecting the agent data, there are several agents that have had hundreds of canceled reservations, and many that have had 100% of their reservations cancel. It is possible they will have an impact on our cancellation forecast, and may be a bit of additional/bonus information we can offer the hotels to show which agents have the most cost or value.

Filling the null values in agent with 0.0.

In [18]:
hotel_df['agent'].fillna(0, inplace=True)

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/2323543621.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hotel_df['agent'].fillna(0, inplace=True)


For company, it would be easy to assume that null values are transient travelers whose reservations are not related to a company, but it is worth a bit more investigation.

In [19]:
pd.crosstab(hotel_df['company'].isnull(), hotel_df['market_segment'])
pd.crosstab(hotel_df['company'].isnull(), hotel_df['distribution_channel'])
pd.crosstab(hotel_df['company'].isnull(), hotel_df['customer_type'])

customer_type,Contract,Group,Transient,Transient-Party
company,,,,
False,5,49,4152,2591
True,4071,528,85461,22533


The company column also appears to have a small number of "true" missing values with 5 Contract and 49 Group reservations showing as null. Since this number is small, and it is not feisable to retrieve the information, filling all null values to show no company makes sense. Similar to the agents, the companies are represented by a float value, so I am filling them with 0.0. 

In [20]:
hotel_df['company'].unique()

array([ nan, 110., 113., 270., 178., 240., 154., 144., 307., 268.,  59.,
       204., 312., 318.,  94., 174., 274., 195., 223., 317., 281., 118.,
        53., 286.,  12.,  47., 324., 342., 373., 371., 383.,  86.,  82.,
       218.,  88.,  31., 397., 392., 405., 331., 367.,  20.,  83., 416.,
        51., 395., 102.,  34.,  84., 360., 394., 457., 382., 461., 478.,
       386., 112., 486., 421.,   9., 308., 135., 224., 504., 269., 356.,
       498., 390., 513., 203., 263., 477., 521., 169., 515., 445., 337.,
       251., 428., 292., 388., 130., 250., 355., 254., 543., 531., 528.,
        62., 120.,  42.,  81., 116., 530., 103.,  39.,  16.,  92.,  61.,
       501., 165., 291., 290.,  43., 325., 192., 108., 200., 465., 287.,
       297., 490., 482., 207., 282., 437., 225., 329., 272.,  28.,  77.,
       338.,  72., 246., 319., 146., 159., 380., 323., 511., 407., 278.,
        80., 403., 399.,  14., 137., 343., 346., 347., 349., 289., 351.,
       353.,  54.,  99., 358., 361., 362., 366., 37

In [21]:
hotel_df['company'].fillna(0, inplace=True)

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3789684405.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hotel_df['company'].fillna(0, inplace=True)


In [22]:
hotel_df.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

## Duplicates

Inspecting and dropping duplicates. While there is not a full proof way to know that these are 100% duplicates due to a lack of unique customer or reservation id's, it would likely have a more negative impact on our ML Model to mistakenly include duplicates than it would be to mistakenly omit some data. For this reason I will drop any rows that are exact duplicates.

In [23]:
hotel_df.duplicated().sum()

31994

In [24]:
hotel_df[hotel_df.duplicated()]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.00,0,1,Check-Out,2015-07-03
22,Resort Hotel,0,72,2015,July,27,1,2,4,2,...,No Deposit,250.0,0.0,0,Transient,84.67,0,1,Check-Out,2015-07-07
43,Resort Hotel,0,70,2015,July,27,2,2,3,2,...,No Deposit,250.0,0.0,0,Transient,137.00,0,1,Check-Out,2015-07-07
138,Resort Hotel,1,5,2015,July,28,5,1,0,2,...,No Deposit,240.0,0.0,0,Transient,97.00,0,0,Canceled,2015-07-01
200,Resort Hotel,0,0,2015,July,28,7,0,1,1,...,No Deposit,240.0,0.0,0,Transient,109.80,0,3,Check-Out,2015-07-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119349,City Hotel,0,186,2017,August,35,31,0,3,2,...,No Deposit,9.0,0.0,0,Transient,126.00,0,2,Check-Out,2017-09-03
119352,City Hotel,0,63,2017,August,35,31,0,3,3,...,No Deposit,9.0,0.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119353,City Hotel,0,63,2017,August,35,31,0,3,3,...,No Deposit,9.0,0.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03
119354,City Hotel,0,63,2017,August,35,31,0,3,3,...,No Deposit,9.0,0.0,0,Transient-Party,195.33,0,2,Check-Out,2017-09-03


In [25]:
cleaned_hotel_df = hotel_df.drop_duplicates()

## Cleaning Data

To preserve the original data, I dropped duplicates into a new dataframe called cleaned_hotel_df. Next I will continue to clean and prepare this dataframe to merge with weather data.

In [26]:
cleaned_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87396 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87396 non-null  object 
 1   is_canceled                     87396 non-null  int64  
 2   lead_time                       87396 non-null  int64  
 3   arrival_date_year               87396 non-null  int64  
 4   arrival_date_month              87396 non-null  object 
 5   arrival_date_week_number        87396 non-null  int64  
 6   arrival_date_day_of_month       87396 non-null  int64  
 7   stays_in_weekend_nights         87396 non-null  int64  
 8   stays_in_week_nights            87396 non-null  int64  
 9   adults                          87396 non-null  int64  
 10  children                        87396 non-null  float64
 11  babies                          87396 non-null  int64  
 12  meal                            8739

In [28]:
# Converting columns that are unnecessarly floats to int.

float_cols = ['children', 'agent', 'company']
cleaned_hotel_df[float_cols] = cleaned_hotel_df[float_cols].astype(int)

cleaned_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87396 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87396 non-null  object 
 1   is_canceled                     87396 non-null  int64  
 2   lead_time                       87396 non-null  int64  
 3   arrival_date_year               87396 non-null  int64  
 4   arrival_date_month              87396 non-null  object 
 5   arrival_date_week_number        87396 non-null  int64  
 6   arrival_date_day_of_month       87396 non-null  int64  
 7   stays_in_weekend_nights         87396 non-null  int64  
 8   stays_in_week_nights            87396 non-null  int64  
 9   adults                          87396 non-null  int64  
 10  children                        87396 non-null  int64  
 11  babies                          87396 non-null  int64  
 12  meal                            8739

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/2839369173.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df[float_cols] = cleaned_hotel_df[float_cols].astype(int)


In [29]:
# Add location information for the hotels to prepare to incorporate weather data

cleaned_hotel_df['hotel_city'] = cleaned_hotel_df['hotel'].map({'City Hotel': 'Lisbon, Portugal', 'Resort Hotel': 'Faro, Portugal'})

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3268563905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['hotel_city'] = cleaned_hotel_df['hotel'].map({'City Hotel': 'Lisbon, Portugal', 'Resort Hotel': 'Faro, Portugal'})


In [30]:
cleaned_hotel_df.sample()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,hotel_city
109976,City Hotel,0,4,2017,April,16,17,1,2,2,...,14,0,0,Transient,164.0,0,1,Check-Out,2017-04-20,"Lisbon, Portugal"


In [34]:
# Converting date columns to datetime

cleaned_hotel_df['arrival_date_month'] = cleaned_hotel_df['arrival_date_month'].str.upper()
cleaned_hotel_df['arrival_date'] = pd.to_datetime(
    cleaned_hotel_df['arrival_date_year'].astype(str) + '-' +
    cleaned_hotel_df['arrival_date_month'] + '-' +
    cleaned_hotel_df['arrival_date_day_of_month'].astype(str),
    format = '%Y-%B-%d', errors='coerce')

cleaned_hotel_df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month', 'arrival_date']].head()

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3662200369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['arrival_date_month'] = cleaned_hotel_df['arrival_date_month'].str.upper()
/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3662200369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['arrival_date'] = pd.to_datetime(


,arrival_date_year,arrival_date_month,arrival_date_day_of_month,arrival_date
0,2015,JULY,1,2015-07-01
1,2015,JULY,1,2015-07-01
2,2015,JULY,1,2015-07-01
3,2015,JULY,1,2015-07-01
4,2015,JULY,1,2015-07-01


In [35]:
cleaned_hotel_df['reservation_status_date'] = pd.to_datetime(cleaned_hotel_df['reservation_status_date'], errors='coerce')

/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3065171112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['reservation_status_date'] = pd.to_datetime(cleaned_hotel_df['reservation_status_date'], errors='coerce')


In [37]:
cleaned_hotel_df[['arrival_date', 'reservation_status_date']].isnull().sum()

arrival_date               0
reservation_status_date    0
dtype: int64

In [38]:
cleaned_hotel_df.shape

(87396, 34)

## Weather Data

Now that the hotel data has been wrangled and cleaned, it is time to work on gathering the weather data that will be needed. The API has has daily and hourly limits, and 87k API calls will far exceed the limits. So I will need to start by batching the data prior to making the calls. I will be looking for two types of weather data - 1. The weather forcast for the arrival date as of the reservation status date for all canceled reservations. This will give insight to if the forecast may have impacted the guest's decision to cancel. 2. The actual weather for the arrival date. This may give us insight as to what the forcasts may have been prior to the guests arrival, therefore telling us if those forecasts had an impact on not canceling their resevation

In [49]:
# Creating a dictionary of the coordinates for the hotels

hotel_coords = {
    "City Hotel": (38.72225, -9.13934),
    "Resort Hotel": (37.01936, -7.93044)}

In [48]:
# Batching Dates for API Calls

unique_arrival_dates = cleaned_hotel_df[['hotel', 'arrival_date']].drop_duplicates()

# Gathering reservation status dates for canceled reservations only
canceled_df = cleaned_hotel_df[cleaned_hotel_df['is_canceled'] == 1].copy()
unique_canceled_dates = canceled_df[['hotel', 'reservation_status_date']].drop_duplicates()
unique_canceled_arr_res_dates = canceled_df[['hotel', 'arrival_date', 'reservation_status_date']].drop_duplicates()

print("Total number of arrival dates: ", len(unique_arrival_dates))

print("Total number of canceled dates: ", len(unique_canceled_dates))

print("Total number of canceled res and arrival dates combined: ", len(unique_canceled_arr_res_dates))


Total number of arrival dates:  1586
Total number of canceled dates:  1710
Total number of canceled res and arrival dates combined:  19027


In [51]:
# calculating the number of days prior to arrival each reservation was canceled

unique_canceled_arr_res_dates['cancel_lead_time'] = (unique_canceled_arr_res_dates['arrival_date'] - unique_canceled_arr_res_dates['reservation_status_date']).dt.days

In [53]:
unique_canceled_arr_res_dates = unique_canceled_arr_res_dates[unique_canceled_arr_res_dates['cancel_lead_time'] >= 0].reset_index(drop=True)

In [54]:
len(unique_canceled_arr_res_dates), unique_canceled_arr_res_dates['cancel_lead_time'].describe()

(19027,
 count    19027.000000
 mean        66.948179
 std         72.216231
 min          0.000000
 25%         13.000000
 50%         40.000000
 75%         99.000000
 max        526.000000
 Name: cancel_lead_time, dtype: float64)

### Weather Data Import

Due to severe api limitations with the original data source, I have decided to import the weather data from Open-Meteo. The next several steps will be to import the weather forecast for cancelled reservations arrival dates as of the cancellation date.

In [66]:
!pip install openmeteo-requests requests-cache retry-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 675.4/675.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [67]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd

In [68]:
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [79]:
# Pick the first canceled reservation for testing
test_row = unique_canceled_arr_res_dates.iloc[0]

hotel = test_row['hotel']
arrival_date = test_row['arrival_date']
reservation_status_date = test_row['reservation_status_date']

lat, lon = hotel_coords[hotel]

params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": reservation_status_date.strftime("%Y-%m-%d"),
    "end_date": reservation_status_date.strftime("%Y-%m-%d"),
    "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "weathercode"],
    "timezone": "auto"
}

responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
response = responses[0]

daily = response.Daily()
print("Temp Max:", daily.Variables(0).ValuesAsNumpy()[0])
print("Temp Min:", daily.Variables(1).ValuesAsNumpy()[0])
print("Precipitation:", daily.Variables(2).ValuesAsNumpy()[0])


Temp Max: 22.205
Temp Min: 13.055
Precipitation: 0.0


In [82]:
weather_records = []

for idx, row in unique_canceled_arr_res_dates.iterrows():
    hotel = row['hotel']
    arrival_date = row['arrival_date']
    reservation_status_date = row['reservation_status_date']

    lat, lon = hotel_coords[hotel]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": reservation_status_date.strftime("%Y-%m-%d"),
        "end_date": reservation_status_date.strftime("%Y-%m-%d"),
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "weathercode"],
        "timezone": "auto"
    }

    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        daily = response.Daily()

        weather_records.append({
            "hotel": hotel,
            "arrival_date": arrival_date,
            "reservation_status_date": reservation_status_date,
            "temp_max": daily.Variables(0).ValuesAsNumpy()[0],
            "temp_min": daily.Variables(1).ValuesAsNumpy()[0],
            "precip_sum": daily.Variables(2).ValuesAsNumpy()[0],
            "weathercode": daily.Variables(3).ValuesAsNumpy()[0]
        })
    except Exception as e:
        print(f"Error for {hotel}, {reservation_status_date}: {e}")


In [83]:
weather_df = pd.DataFrame(weather_records)
weather_df.head()

,hotel,arrival_date,reservation_status_date,temp_max,temp_min,precip_sum,weathercode
0,Resort Hotel,2015-07-01,2015-05-06,22.205000,13.055000,0.0,3.0
1,Resort Hotel,2015-07-01,2015-04-22,20.855000,15.205000,0.2,51.0
2,Resort Hotel,2015-07-01,2015-06-23,26.655001,17.705000,0.0,1.0
3,Resort Hotel,2015-07-01,2015-05-11,24.855000,18.605000,0.0,1.0
4,Resort Hotel,2015-07-01,2015-05-29,25.155001,18.754999,0.2,51.0


The weathercode indicates the weather description, which will likely be the most important correlation for weather to cancellations. Creating a mapping dictionary so we can see the descriptions.

In [84]:
weather_mapping = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    71: "Slight snowfall",
    73: "Moderate snowfall",
    75: "Heavy snowfall",
    80: "Rain showers",
    81: "Moderate rain showers",
    82: "Violent rain showers",
    95: "Thunderstorm",
    99: "Thunderstorm with hail"
}

In [85]:
weather_df["weather_description"] = weather_df["weathercode"].map(weather_mapping)

In [86]:
weather_df.head()

,hotel,arrival_date,reservation_status_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,2015-07-01,2015-05-06,22.205000,13.055000,0.0,3.0,Overcast
1,Resort Hotel,2015-07-01,2015-04-22,20.855000,15.205000,0.2,51.0,Light drizzle
2,Resort Hotel,2015-07-01,2015-06-23,26.655001,17.705000,0.0,1.0,Mainly clear
3,Resort Hotel,2015-07-01,2015-05-11,24.855000,18.605000,0.0,1.0,Mainly clear
4,Resort Hotel,2015-07-01,2015-05-29,25.155001,18.754999,0.2,51.0,Light drizzle


In [87]:
weather_df.isna().sum()

hotel                      0
arrival_date               0
reservation_status_date    0
temp_max                   0
temp_min                   0
precip_sum                 0
weathercode                0
weather_description        0
dtype: int64

In [88]:
weather_df.shape

(19027, 8)

Now that I have pulled the most important weather data, forecasts for arrival days of canceled reservations made on the cancellation date, I think in later steps of this project I may want to compare these forecasts to forecasts that guests would have seen on reservations that did NOT cancel. To do this, I will find the median 'cancel_lead_time' which will show me the most common number of days prior to arrival that reservations cancel, then I will find the weather forecasts for those dates.

In [89]:
# Ensure datetime columns are datetime objects
cleaned_hotel_df['arrival_date'] = pd.to_datetime(cleaned_hotel_df['arrival_date'])
cleaned_hotel_df['reservation_status_date'] = pd.to_datetime(cleaned_hotel_df['reservation_status_date'])

# For canceled reservations only
canceled_df = cleaned_hotel_df[cleaned_hotel_df['is_canceled'] == 1].copy()
canceled_df['cancel_lead_time'] = (canceled_df['arrival_date'] - canceled_df['reservation_status_date']).dt.days

median_cancel_lead = canceled_df['cancel_lead_time'].median()
print("Median cancellation lead time (days):", median_cancel_lead)


Median cancellation lead time (days): 35.0


/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3903066115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['arrival_date'] = pd.to_datetime(cleaned_hotel_df['arrival_date'])
/var/folders/vn/_qrvqjjx1yb4mwjbsrg0tnj80000gn/T/ipykernel_36628/3903066115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hotel_df['reservation_status_date'] = pd.to_datetime(cleaned_hotel_df['reservation_status_date'])


In [90]:
non_canceled_df = cleaned_hotel_df[cleaned_hotel_df['is_canceled'] == 0].copy()
non_canceled_df['forecast_date'] = non_canceled_df['arrival_date'] - pd.to_timedelta(median_cancel_lead, unit='d')

In [92]:
non_canceled_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,hotel_city,arrival_date,forecast_date
0,Resort Hotel,0,342,2015,JULY,27,1,0,0,2,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27
1,Resort Hotel,0,737,2015,JULY,27,1,0,0,2,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27
2,Resort Hotel,0,7,2015,JULY,27,1,0,1,1,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27
3,Resort Hotel,0,13,2015,JULY,27,1,0,1,1,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27
4,Resort Hotel,0,14,2015,JULY,27,1,0,2,2,...,0,Transient,98.0,0,1,Check-Out,2015-07-03,"Faro, Portugal",2015-07-01,2015-05-27


In [95]:
unique_noncanceled_arr_for_dates = non_canceled_df[['hotel', 'arrival_date', 'forecast_date']].drop_duplicates()

print(len(unique_noncanceled_arr_for_dates))

1581


In [96]:
unique_noncanceled_arr_for_dates.shape


(1581, 3)

In [97]:
unique_noncanceled_arr_for_dates.head()

,hotel,arrival_date,forecast_date
0,Resort Hotel,2015-07-01,2015-05-27
33,Resort Hotel,2015-07-02,2015-05-28
65,Resort Hotel,2015-07-03,2015-05-29
100,Resort Hotel,2015-07-04,2015-05-30
131,Resort Hotel,2015-07-05,2015-05-31


In [98]:
non_canceled_weather_records = []

for idx, row in unique_noncanceled_arr_for_dates.iterrows():
    hotel = row['hotel']
    arrival_date = row['arrival_date']
    forecast_date = row['forecast_date']

    lat, lon = hotel_coords[hotel]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": forecast_date.strftime("%Y-%m-%d"),
        "end_date": forecast_date.strftime("%Y-%m-%d"),
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "weathercode"],
        "timezone": "auto"
    }

    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        daily = response.Daily()

        non_canceled_weather_records.append({
            "hotel": hotel,
            "arrival_date": arrival_date,
            "forecast_date": forecast_date,
            "temp_max": daily.Variables(0).ValuesAsNumpy()[0],
            "temp_min": daily.Variables(1).ValuesAsNumpy()[0],
            "precip_sum": daily.Variables(2).ValuesAsNumpy()[0],
            "weathercode": daily.Variables(3).ValuesAsNumpy()[0],
            "weather_description": weather_mapping.get(daily.Variables(3).ValuesAsNumpy()[0], "Unknown")
        })
    except Exception as e:
        print(f"Error for {hotel}, {forecast_date}: {e}")


In [99]:
non_canceled_weather_df = pd.DataFrame(non_canceled_weather_records)
print(non_canceled_weather_df.head())

          hotel arrival_date forecast_date   temp_max   temp_min  precip_sum  \
0  Resort Hotel   2015-07-01    2015-05-27  26.754999  19.305000         0.0   
1  Resort Hotel   2015-07-02    2015-05-28  28.055000  18.955000         0.0   
2  Resort Hotel   2015-07-03    2015-05-29  25.155001  18.754999         0.2   
3  Resort Hotel   2015-07-04    2015-05-30  27.405001  16.605000         0.0   
4  Resort Hotel   2015-07-05    2015-05-31  28.305000  18.055000         0.0   

   weathercode weather_description  
0          0.0           Clear sky  
1          2.0       Partly cloudy  
2         51.0       Light drizzle  
3          0.0           Clear sky  
4          0.0           Clear sky  


Now with weather data for both canceled reservation dates and non-canceled reservation dates have been imported, it is time to combine all the information into one data frame

In [133]:
combined_df = cleaned_hotel_df.copy()

In [101]:
weather_df.head()

,hotel,arrival_date,reservation_status_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,2015-07-01,2015-05-06,22.205000,13.055000,0.0,3.0,Overcast
1,Resort Hotel,2015-07-01,2015-04-22,20.855000,15.205000,0.2,51.0,Light drizzle
2,Resort Hotel,2015-07-01,2015-06-23,26.655001,17.705000,0.0,1.0,Mainly clear
3,Resort Hotel,2015-07-01,2015-05-11,24.855000,18.605000,0.0,1.0,Mainly clear
4,Resort Hotel,2015-07-01,2015-05-29,25.155001,18.754999,0.2,51.0,Light drizzle


In [104]:
weather_df.dtypes

hotel                              object
arrival_date               datetime64[ns]
reservation_status_date    datetime64[ns]
temp_max                          float32
temp_min                          float32
precip_sum                        float32
weathercode                       float32
weather_description                object
dtype: object

In [109]:
canceled_res_df = combined_df[combined_df['is_canceled'] == 1].copy()

In [110]:
print("Weather DF: ", weather_df.shape)
print("Canceled Res DF: ", canceled_res_df.shape)

Weather DF:  (19027, 8)
Canceled Res DF:  (24025, 34)


In [111]:
canceled_res_with_weather = canceled_res_df.merge(
    weather_df,
    on=['hotel', 'arrival_date', 'reservation_status_date'],
    how='left',
    suffixes=('', '_weather')
)

In [112]:
canceled_res_with_weather.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,total_of_special_requests,reservation_status,reservation_status_date,hotel_city,arrival_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,1,85,2015,JULY,27,1,0,3,2,...,1,Canceled,2015-05-06,"Faro, Portugal",2015-07-01,22.205000,13.055000,0.0,3.0,Overcast
1,Resort Hotel,1,75,2015,JULY,27,1,0,3,2,...,0,Canceled,2015-04-22,"Faro, Portugal",2015-07-01,20.855000,15.205000,0.2,51.0,Light drizzle
2,Resort Hotel,1,23,2015,JULY,27,1,0,4,2,...,0,Canceled,2015-06-23,"Faro, Portugal",2015-07-01,26.655001,17.705000,0.0,1.0,Mainly clear
3,Resort Hotel,1,60,2015,JULY,27,1,2,5,2,...,2,Canceled,2015-05-11,"Faro, Portugal",2015-07-01,24.855000,18.605000,0.0,1.0,Mainly clear
4,Resort Hotel,1,96,2015,JULY,27,1,2,8,2,...,2,Canceled,2015-05-29,"Faro, Portugal",2015-07-01,25.155001,18.754999,0.2,51.0,Light drizzle


In [113]:
canceled_res_with_weather.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

In [116]:
canceled_res_with_weather.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,total_of_special_requests,reservation_status,reservation_status_date,hotel_city,arrival_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,1,85,2015,JULY,27,1,0,3,2,...,1,Canceled,2015-05-06,"Faro, Portugal",2015-07-01,22.205000,13.055000,0.0,3.0,Overcast
1,Resort Hotel,1,75,2015,JULY,27,1,0,3,2,...,0,Canceled,2015-04-22,"Faro, Portugal",2015-07-01,20.855000,15.205000,0.2,51.0,Light drizzle
2,Resort Hotel,1,23,2015,JULY,27,1,0,4,2,...,0,Canceled,2015-06-23,"Faro, Portugal",2015-07-01,26.655001,17.705000,0.0,1.0,Mainly clear
3,Resort Hotel,1,60,2015,JULY,27,1,2,5,2,...,2,Canceled,2015-05-11,"Faro, Portugal",2015-07-01,24.855000,18.605000,0.0,1.0,Mainly clear
4,Resort Hotel,1,96,2015,JULY,27,1,2,8,2,...,2,Canceled,2015-05-29,"Faro, Portugal",2015-07-01,25.155001,18.754999,0.2,51.0,Light drizzle


In [120]:
# add forecast_Date column to canceled reservations equal to reservation status date.

canceled_res_with_weather['forecast_date'] = canceled_res_with_weather['reservation_status_date']
canceled_res_with_weather.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reservation_status,reservation_status_date,hotel_city,arrival_date,temp_max,temp_min,precip_sum,weathercode,weather_description,forecast_date
0,Resort Hotel,1,85,2015,JULY,27,1,0,3,2,...,Canceled,2015-05-06,"Faro, Portugal",2015-07-01,22.205000,13.055000,0.0,3.0,Overcast,2015-05-06
1,Resort Hotel,1,75,2015,JULY,27,1,0,3,2,...,Canceled,2015-04-22,"Faro, Portugal",2015-07-01,20.855000,15.205000,0.2,51.0,Light drizzle,2015-04-22
2,Resort Hotel,1,23,2015,JULY,27,1,0,4,2,...,Canceled,2015-06-23,"Faro, Portugal",2015-07-01,26.655001,17.705000,0.0,1.0,Mainly clear,2015-06-23
3,Resort Hotel,1,60,2015,JULY,27,1,2,5,2,...,Canceled,2015-05-11,"Faro, Portugal",2015-07-01,24.855000,18.605000,0.0,1.0,Mainly clear,2015-05-11
4,Resort Hotel,1,96,2015,JULY,27,1,2,8,2,...,Canceled,2015-05-29,"Faro, Portugal",2015-07-01,25.155001,18.754999,0.2,51.0,Light drizzle,2015-05-29


In [131]:
non_canceled_weather_df.head()

,hotel,arrival_date,forecast_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,2015-07-01,2015-05-27,26.754999,19.305000,0.0,0.0,Clear sky
1,Resort Hotel,2015-07-02,2015-05-28,28.055000,18.955000,0.0,2.0,Partly cloudy
2,Resort Hotel,2015-07-03,2015-05-29,25.155001,18.754999,0.2,51.0,Light drizzle
3,Resort Hotel,2015-07-04,2015-05-30,27.405001,16.605000,0.0,0.0,Clear sky
4,Resort Hotel,2015-07-05,2015-05-31,28.305000,18.055000,0.0,0.0,Clear sky


In [137]:
non_canceled_res_copy = combined_df[combined_df['is_canceled'] == 0].copy()

In [139]:
non_canceled_res_df = non_canceled_res_copy.merge(non_canceled_weather_df,
                                                  on=['hotel', 'arrival_date'],
                                                  how='left',
                                                  suffixes=('', '_weather'))

In [140]:
non_canceled_res_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reservation_status,reservation_status_date,hotel_city,arrival_date,forecast_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,0,342,2015,JULY,27,1,0,0,2,...,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
1,Resort Hotel,0,737,2015,JULY,27,1,0,0,2,...,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
2,Resort Hotel,0,7,2015,JULY,27,1,0,1,1,...,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
3,Resort Hotel,0,13,2015,JULY,27,1,0,1,1,...,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
4,Resort Hotel,0,14,2015,JULY,27,1,0,2,2,...,Check-Out,2015-07-03,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky


In [141]:
# Combine the canceled and non-canceled dataframes including weather data

hotel_weather_df = pd.concat([non_canceled_res_df, canceled_res_with_weather], ignore_index=True)

In [142]:
hotel_weather_df.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

In [143]:
hotel_weather_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reservation_status,reservation_status_date,hotel_city,arrival_date,forecast_date,temp_max,temp_min,precip_sum,weathercode,weather_description
0,Resort Hotel,0,342,2015,JULY,27,1,0,0,2,...,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
1,Resort Hotel,0,737,2015,JULY,27,1,0,0,2,...,Check-Out,2015-07-01,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
2,Resort Hotel,0,7,2015,JULY,27,1,0,1,1,...,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
3,Resort Hotel,0,13,2015,JULY,27,1,0,1,1,...,Check-Out,2015-07-02,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky
4,Resort Hotel,0,14,2015,JULY,27,1,0,2,2,...,Check-Out,2015-07-03,"Faro, Portugal",2015-07-01,2015-05-27,26.754999,19.305,0.0,0.0,Clear sky


In [144]:
hotel_weather_df.to_csv('hotel_weather_cleaned.csv', index=False)

## Data Wrangling Conclusion

In this project I imported, cleaned, and prepared the data from the Hotel Bookings Data Set, then used that information to create variables to pull weather data through an API. I pulled weather forecasts for canceled reservations, as well as weather forecasts for non-canceled reservations, cleaned and prepared them into separate dataframes, then combined all dataframes into one.

At the end of this project, we have a clean dataframe called hotel_weather_df that is ready for the next steps of this CapStone project.